In [1]:
import pandas as pd
import numpy as np
from time import process_time

In [2]:
parameters = {
    "n": 50,  # population size
    "p_const_heur":0.6, # ratio of initial pop.with construction heuristic solns
    "p_m": 0.3,  # mutation probabibility (not used)
    "p": 10,  # selection parent size (even number)
    "t": 3,  # tournament size
    "p_elit": 0.7,  # ratio of elite individuals in replacement,
    "n_iter": 5000,  # max number of iterations
    "n_non_improv_iter" : 400, # max number of iter without improvement
    "log_mode":True, # in log mode it checks solution validity
    "problem": "eil51",  # problem name. eil51, eil76 or eil101
    "profit":"Low"  # High or Low
}

In [3]:
# read data from excel
def read_data():
    df_data = pd.read_excel("./dataset-TSPP.xlsx",sheet_name=parameters["problem"],index_col=0)
    df_data = df_data[["x","y",parameters["profit"]]]
    return df_data

In [4]:
# find distance btw two location
def find_distances(row,df_data):
    d = np.sqrt((df_data["x"]-row["x"])**2 + (df_data["y"]-row["y"])**2)
    return d

In [5]:
# create distance matrix (distance for each location pair)
def create_distance_matrix(df_data):
    df_distance = df_data.apply(find_distances,axis=1,df_data=df_data)
    df_distance = np.round(df_distance, 2)
    return df_distance

In [6]:
# given visit order, it finds fitness (objective value) of a solution
def calculate_fitness(df_distance,df_profit,visit_order):
    prev_cities =  visit_order[np.arange(visit_order.size)-1]
    total_distance = df_distance.values[visit_order,prev_cities].sum()
    total_profit =  df_profit.loc[visit_order].sum()
    return np.round(total_profit-total_distance,2)     

In [7]:
# checks whether there are repeated locations and if depot is in the tour
def verify_soln(soln):
    u, c = np.unique(soln["VisitOrder"], return_counts=True)
    feasible = np.all(c==1) and np.any(soln["VisitOrder"] == 0)
    if not feasible:
        print("Soln infeasible:",soln)

In [8]:
# given visit order it creates solution object
# solution object includes visit order,no of cities and also fitness value
def create_solution(df_distance,df_profit,visit_order):
    fitness = calculate_fitness(df_distance,df_profit,visit_order)
    soln = {"Fitness":fitness,"VisitOrder":visit_order, "NoOfCities":visit_order.size }
    if parameters["log_mode"]:
        verify_soln(soln)
    return soln

In [9]:
# it creates totally random solutions i.e. random tour length and random visit order
def create_random_solution(df_distance,df_profit):
    tour_length = np.random.randint(low=1,high=df_distance.shape[0]) #random tour length
    cities = np.random.choice(df_distance.index[1:], size=tour_length, replace=False) # random order
    visit_order = np.concatenate([np.array([0]),cities])  # add depot
    return create_solution(df_distance,df_profit,visit_order)
    

In [10]:
# it creates partially random solutions. The tour length and customers are random
# but the visit order is determined by greedy approach i.e. always visit nearist customer

def create_partially_random_soln(df_distance,df_profit,tour_length=None):
    if tour_length == None:
        tour_length = np.random.randint(low=1,high=df_distance.shape[0]) #random tour length
    cities = np.random.choice(df_distance.index[1:], size=tour_length, replace=False) # random cities
    cities = np.concatenate([np.array([0]),cities])  # add depot
    cities_in_tour = cities.tolist()
    
    # visit cities by greedy approach
    tour =  []
    current_city = cities_in_tour[np.random.randint(len(cities_in_tour))]
    while len(cities_in_tour)>1:
        tour.append(current_city)
        cities_in_tour.remove(current_city)
        current_city =  df_distance[current_city].loc[cities_in_tour].idxmin()
    # add last city
    tour.append(current_city)
    visit_order = np.array(tour)
    return create_solution(df_distance,df_profit,visit_order)
    

In [11]:
# Creates solutions by constructive heuristics
# no_of_solns: number of solutions to be generated
# tour_lengths: an array indicating the tour length.
# no_of_solns is distributed to different tour lengths evenly.
# Construction heuristic: 
#  - Give a goodness score for each customer by subtracting average distance
#    to other customers from profit.
#  - Take best k customers (k = tour length) 
#  - By starting from a random customer, visit cities by greedy approach.

def solns_from_contruction_heuristic(df_distance,df_profit,tour_lengths, no_of_solns):
    k = int(np.ceil(no_of_solns/len(tour_lengths)))  # no_of_soln for each tourlength
    df_city_score = df_profit[1:] - df_distance.loc[1:].mean(axis=1) # goodness score for each city
    all_solns = []
    
    for tour_len in tour_lengths:
        for i in range(k):
            cities_in_tour = df_city_score.sort_values(ascending=False).index[:tour_len]
            cities_in_tour = [0]+cities_in_tour.to_list()
            # visit cities by greedy approach
            tour =  []
            current_city = cities_in_tour[np.random.randint(len(cities_in_tour))]
            while len(cities_in_tour)>1:
                tour.append(current_city)
                cities_in_tour.remove(current_city)
                current_city =  df_distance[current_city].loc[cities_in_tour].idxmin()
            # add last city
            tour.append(current_city)
            visit_order = np.array(tour)
            all_solns.append(create_solution(df_distance,df_profit,visit_order))
    return all_solns

In [12]:
# This method takes a solution and tries to improve it by removing
# customers that dont contribute to objective score i.e. removing this customer
# will increase objective value. It is determined by considering it's length to two
# neighboring customers in the tour, it's profit and the length btw its two neighboring
# customers in the tour. 

def update_soln(offspring,df_distance,df_profit):
    visit_order = offspring.VisitOrder
    
    while True:
        prev_cities =  visit_order[np.arange(visit_order.size)-1]
        next_indices = (np.arange(visit_order.size)+1)
        next_indices[next_indices.size-1] = 0
        next_cities = visit_order[next_indices]

        # for each city, find fitness change after removing from tour
        dist_prev = df_distance.values[visit_order,prev_cities]
        dist_next = df_distance.values[visit_order,next_cities]
        dist_prev_next = df_distance.values[prev_cities,next_cities]
        profits = df_profit.loc[visit_order].values 
        changes = dist_prev  + dist_next - dist_prev_next - profits
        if np.any(changes > 0) and visit_order[changes.argmax()]!=0: # dont remove depot
            visit_order = np.delete(visit_order, changes.argmax())
        else:
            break
    
    return pd.Series(create_solution(df_distance,df_profit,visit_order)) 

In [13]:
# Creates initial population from construction heuristic and partially random solutions.
# Their ratio in population is determined by "p_const_heur" parameter.
# For the tour length of construction heuristic solutions we give 5,10,15... until number of 
# all customers to obtain divergence.

def create_initial_population(df_distance,df_profit):
    
    # create from const. heuristic
    tour_lengths = np.arange(df_distance.shape[0],step = 5)[1:]
    heuristic_solns = solns_from_contruction_heuristic(df_distance,df_profit,tour_lengths, no_of_solns = int(parameters["n"]*parameters["p_const_heur"]))
    
    # create partially random solns
    rand_solns = []
    for i in range(parameters["n"]-len(heuristic_solns)):
        rand_solns.append(create_partially_random_soln(df_distance,df_profit))
    
    all_solns = pd.DataFrame(heuristic_solns + rand_solns)
    return all_solns

In [14]:
# tournament selection procedure is applied to select parents
# each time a pool of solutions are selected randomly and the best one is 
# put into parent list.
def selection(all_solns):
    parent_indices =  []
    for i in range(parameters["p"]):
        rivals = np.random.choice(all_solns.index, size=parameters["t"], replace=False)
        parent_indices.append(all_solns.loc[rivals].sort_values("Fitness",ascending=False).index[0])
    return all_solns.loc[parent_indices]

In [15]:
# Order cross over (OX) is used. 
# It is a crossover operator specific to permutation solution representation.
# The details of the operator is provided in the report.
# P.S.The sort and long parent information was intended to use
# for different reason but not used afterwards. 

def cross_over(p1,p2,df_distance, df_profit):
    # find short,long tours 
    if p1.NoOfCities < p2.NoOfCities:
        p_short = p1
        p_long = p2
    else:
        p_short = p2
        p_long = p1
    
    # apply order crossover to find o1
    two_points = sorted(np.random.choice(np.arange(p_short.NoOfCities), size=2, replace=False))
    middle_section = p_short.VisitOrder[two_points[0]:two_points[1]+1]
    other_sections= [c for c in p_long.VisitOrder if c not in middle_section]
    half_len = int(len(other_sections)/2)
    o1_visit_order =  np.array(other_sections[:half_len] + middle_section.tolist() + other_sections[half_len:])
    o1 = create_solution(df_distance, df_profit, o1_visit_order)
    
    # apply order crossover to find o2
    two_points = sorted(np.random.choice(np.arange(p_long.NoOfCities), size=2, replace=False))
    middle_section = p_long.VisitOrder[two_points[0]:two_points[1]+1]
    other_sections= [c for c in p_short.VisitOrder if c not in middle_section]
    half_len = int(len(other_sections)/2)
    o2_visit_order =  np.array(other_sections[:half_len] + middle_section.tolist() + other_sections[half_len:])
    o2 = create_solution(df_distance, df_profit, o2_visit_order)
    
    return o1,o2

In [16]:
# Generates offsprings from parents. 
# Parent pairs are selected in their order in the list.
def get_offsprings(parents,df_distance, df_profit):
    offsprings = []
    for i in np.arange(parents.shape[0],step=2):
        p1= parents.iloc[i]
        p2= parents.iloc[i+1]
        o1,o2 = cross_over(p1,p2,df_distance, df_profit)
        offsprings += [o1,o2]
    return pd.DataFrame(offsprings) 

In [17]:
# First choose the best solutions (elite solns) with p_elit ratio.
# Then select the other ones randomly from remaining solutions.
# (This is not used in the last version.)
def replacement(current_pop,offsprings):
    all_solns =  pd.concat([current_pop,offsprings],ignore_index = True).sort_values("Fitness",ascending=False)
    n_elite = int(parameters["n"]*parameters["p_elit"])
    elite_solns = all_solns.iloc[:n_elite]
    other_solns_indices = np.random.choice(all_solns.iloc[n_elite:].index, size=parameters["n"]-n_elite, replace=False)
    other_solns = all_solns.loc[other_solns_indices]
    new_pop = pd.concat([elite_solns,other_solns])
    return new_pop

In [18]:
# First, duplicate solutions are eliminated. Then if there are less solution than population
# size, we generate partial random solutions. The remaining procedure is the same
# as "replacement" function.
# (This is not used in the last version.)

def replacementV2(current_pop,offsprings,df_distance,df_profit):
    all_solns =  pd.concat([current_pop,offsprings],ignore_index = True)
    all_solns = all_solns.drop_duplicates(subset='Fitness') # remove repeated solns
    pop_size = all_solns.shape[0] 
    if pop_size < parameters["n"]: # if no of solns less than pop.size, create new ones
        for i in range(parameters["n"]-pop_size):
            all_solns = all_solns.append(create_partially_random_soln(df_distance,df_profit),ignore_index=True)
    all_solns = all_solns.sort_values("Fitness",ascending=False)
    n_elite = int(parameters["n"]*parameters["p_elit"]) # elite solns
    elite_solns = all_solns.iloc[:n_elite]
    other_solns_indices = np.random.choice(all_solns.iloc[n_elite:].index, size=parameters["n"]-n_elite, replace=False) # other solns
    other_solns = all_solns.loc[other_solns_indices]
    new_pop = pd.concat([elite_solns,other_solns])
    return new_pop

In [19]:
# First, duplicate solutions are eliminated. Then if there are less solution than population
# size, we generate partial random solutions.Then we choose best solutions with p_elit ratio.
# The others are extended versions of elite solutions. 

def replacementV3(current_pop,offsprings,df_distance,df_profit):
    all_solns =  pd.concat([current_pop,offsprings],ignore_index = True)
    all_solns = all_solns.drop_duplicates(subset='Fitness') # remove repeated solns
    pop_size = all_solns.shape[0] 
    if pop_size < parameters["n"]: # if no of solns less than pop.size, create new ones
        for i in range(parameters["n"]-pop_size):
            all_solns = all_solns.append(create_partially_random_soln(df_distance,df_profit),ignore_index=True)
    all_solns = all_solns.sort_values("Fitness",ascending=False)
    n_elite = int(parameters["n"]*parameters["p_elit"]) # elite solns
    elite_solns = all_solns.iloc[:n_elite]
    
    # create other solutions by adding cities to some elite solutions
    no_other_solns = parameters["n"]-n_elite
    other_solns = all_solns.loc[np.random.choice(elite_solns.index, size=no_other_solns, replace=False)].copy() # select from elite solns
    for i in range(other_solns.shape[0]):
        soln_visitorder = other_solns.iloc[i].VisitOrder
        out_of_tour_cities = [i for i in range(df_distance.shape[0]) if i not in soln_visitorder]
        closest_cities = df_distance[out_of_tour_cities].loc[soln_visitorder].idxmin() # closest cities
        added_cities = df_distance[out_of_tour_cities].loc[soln_visitorder].min().sort_values()[:5].index # best 5 cities (most closest to cities in tour) 
        soln_visitorder_l = soln_visitorder.tolist()
        for city in added_cities:
            index_of_closest_city = soln_visitorder_l.index(closest_cities[city]) # find loc. of closest city
            soln_visitorder_l.insert(index_of_closest_city, city) # locate there
        elite_solns = elite_solns.append(create_solution(df_distance,df_profit,np.array(soln_visitorder_l)),ignore_index=True)  # add this solution to population
    return elite_solns

In [20]:
# It removes copies of the best solution in the population to increase divergence. 
# It makes this if more than 20% of the solutions are copies of the best solution
# It is called once after each 50 iterations
def clear_copy_solns(current_pop,df_distance,df_profit):
    best_fitness = current_pop.iloc[0].Fitness
    if np.mean(current_pop["Fitness"]==best_fitness) > 0.2:
        copy_indices = current_pop[current_pop["Fitness"]==best_fitness].index[1:]
        current_pop = current_pop[~current_pop.index.isin(copy_indices)]
        n_remaining = parameters["n"] - current_pop.shape[0]
        for i in range(n_remaining):
            current_pop = current_pop.append(create_partially_random_soln(df_distance,df_profit),ignore_index=True)
    return current_pop

In [21]:
def run_ga():
    
    # set seed
    np.random.seed(0)
    
    # read data
    df_data = read_data()
    df_profit =  df_data[parameters["profit"]]
    
    t_start = process_time()
    
    # create distance matrix
    df_distance = create_distance_matrix(df_data)
    
    # find initial pop.
    initial_pop = create_initial_population(df_distance,df_profit)
    
    # update initial pop. to eliminate redundancy
    initial_pop = initial_pop.apply(update_soln,axis = 1, df_distance=df_distance, df_profit=df_profit)
    current_pop = initial_pop
    
    # convergence counter
    conv_counter = 0
    incumbent_fitness = 0
    
    for i in range(parameters["n_iter"]):
        # selection
        parents = selection(current_pop)
        
        # cross over
        offsprings = get_offsprings(parents,df_distance, df_profit)
        offsprings = offsprings.apply(update_soln,axis = 1, df_distance=df_distance, df_profit=df_profit)
        
        # replacement
        current_pop = replacementV3(current_pop,offsprings,df_distance,df_profit)
        
        # clear repeated solns
        if i % 50 == 0:
            current_pop = clear_copy_solns(current_pop,df_distance,df_profit)
        
        # convergence check
        best_fitness = current_pop.iloc[0].Fitness
        if incumbent_fitness == best_fitness:
            conv_counter += 1
            if conv_counter>=parameters["n_non_improv_iter"]:
                break # stop algoithm
        else:
            conv_counter = 0
            incumbent_fitness = best_fitness
        
        
        # log stats
        if i % 20 == 0:
            print("========Pop: ",i,"========")
            print("Best fitness: ",current_pop.iloc[0].Fitness )
            print("Avg fitness: ",current_pop["Fitness"].mean())
            print("Distinct solns: ",current_pop["Fitness"].nunique()) 
            print("Elapsed time: ", process_time()-t_start)
        
    
    print("========Final stats=======")
    print("Total elapsed time: ", process_time()-t_start)
    print("Best fitness: ",current_pop.iloc[0].Fitness )
    print("Avg fitness: ",current_pop["Fitness"].mean())
    print("Distinct solns: ",current_pop["Fitness"].nunique()) 
    return initial_pop,current_pop

In [22]:
initial_pop,current_pop = run_ga()

=======================clear_copy_solns===============================
========Pop:  0 ========
Best fitness:  18.54
Avg fitness:  -9.144599999999999
Distinct solns:  50
Elapsed time:  0.859375
========Pop:  20 ========
Best fitness:  29.19
Avg fitness:  5.9006000000000025
Distinct solns:  47
Elapsed time:  3.359375
========Pop:  40 ========
Best fitness:  29.19
Avg fitness:  12.718599999999999
Distinct solns:  50
Elapsed time:  5.59375
=======================clear_copy_solns===============================
========Pop:  60 ========
Best fitness:  34.52
Avg fitness:  16.1446
Distinct solns:  50
Elapsed time:  7.65625
========Pop:  80 ========
Best fitness:  36.95
Avg fitness:  18.5594
Distinct solns:  50
Elapsed time:  9.75
=======================clear_copy_solns===============================
========Pop:  100 ========
Best fitness:  36.95
Avg fitness:  20.8284
Distinct solns:  50
Elapsed time:  11.953125
========Pop:  120 ========
Best fitness:  41.26
Avg fitness:  22.93
Distinct soln

In [23]:
current_pop.iloc[0]

Fitness                                                   50.74
VisitOrder    [31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...
NoOfCities                                                   22
Name: 0, dtype: object

In [24]:
current_pop.iloc[0].VisitOrder    

array([31,  0, 47,  5, 13, 17,  3, 16, 43, 14, 44, 32,  9, 29, 33, 49, 15,
        8, 37,  4, 45, 50])

In [25]:
current_pop

,Fitness,VisitOrder,NoOfCities
0,50.74,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",22
1,50.25,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",21
2,49.98,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",20
3,49.94,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",22
4,49.49,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",23
5,49.42,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",22
6,49.23,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",21
7,48.88,"[45, 31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, ...",22
8,48.82,"[45, 31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, ...",21
9,48.80,"[31, 0, 47, 5, 13, 17, 3, 16, 43, 14, 44, 32, ...",22
